This notebook taks the CSV as an input. The CSV was obtained from the original Dataset containing all the reviews from the users for the apps on the store. I've applaied a simple pre-processing steps in order to obtain an homogeneous corpus to feed LDA. Once the model was trained I've used it to extract the Topic distribuition latent in every reveiw. The Topic distribuition represents my set of features on which I will train the classifier. This is a Multi-Class classification problem in which I have N possibile classes in which classify my samples and where every sample has a certain amount of features.

In [2]:
import pandas as pd
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn import datasets
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn import metrics  #Import scikit-learn metrics module for accuracy calculation
from sklearn.linear_model import SGDClassifier
from sklearn import linear_model
from sklearn.metrics import classification_report
import numpy as np
import re
  
def strToList(dato):
  line = re.sub('[(,)]', '', dato).split(" ") 
  distribuzione=[]
  for x in range(1, 15):
   if(x%2)!=0:
    distribuzione.append(float(line[x]))
  return distribuzione  



First things first. Let's import the dataset and let's give a peak at the data.

In [3]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/FinalDataset.csv",error_bad_lines=False, sep=',')
data.head() # Shows me the first 5 rows of the data set

#effettuo il drop di tutte quelle righe appartenenti a categorie espresse in modo composto
data = data[data.category != 'Connectivity,Development']
data = data[data.category != 'Connectivity,Theming']
data = data[data.category != 'Internet,Phone & SMS']
data = data[data.category != 'Internet,Reading']
data = data[data.category != 'Internet,Security']
data = data[data.category != 'Multimedia,Graphics']
data = data[data.category != 'Multimedia,Theming']
data = data[data.category != 'Phone & SMS']
data = data[data.category != 'Phone & SMS,Internet']
data = data[data.category != 'Phone & SMS,Multimedia']
data = data[data.category != 'Phone & SMS,Security']
data = data[data.category != 'Reading,Multimedia']
data = data[data.category != 'Science & Education']
data = data[data.category != 'Science & Education,Time']
data = data[data.category != 'Security,Internet']
data = data[data.category != 'Sports & Health']
data = data[data.category != 'System,Time']
data = data[data.category != 'Time,Writing']
data = data[data.category != 'System,Development']
data = data[data.category != 'Money']
data = data[data.category != 'Time']
data = data[data.category != 'Writing']
data = data[data.category != 'Graphics']
data = data[data.category != 'Development']

#converto i nomi delle classi rimanenti label numeriche



data.head()

,review,category,TopicDistribuition
97,Crippling power consumption The devs really ne...,Internet,"[(0, 0.020833334), (1, 0.5208332), (2, 0.02083..."
98,Unusable in large rooms No way to filter out j...,Internet,"[(0, 0.0625), (1, 0.0625), (2, 0.0625), (3, 0...."
99,is there a way to hide channel list when using...,Internet,"[(0, 0.03125), (1, 0.53114474), (2, 0.03125), ..."
100,+1 star for finally including auto-reconnect ...,Internet,"[(0, 0.008928572), (1, 0.232126), (2, 0.366071..."
101,Material design update very nice,Internet,"[(0, 0.3750001), (1, 0.041666668), (2, 0.04166..."


I don't need the 'Review' column anymore. I have to extract 'category' and 'TopicDistribuition' column. I start extracting 'TopicDistribuition' first since I need to convert each string into a probability distribuition. So I took the 'TopicDistribuitions' column and transfer it into a list called 'temp'. 'temp' is a list of strings where each string represent a probability distribuition. I need just the numbers, so I need to convert each string.

In [4]:
temp=data.TopicDistribuition.tolist()
print("Converto la colonna Topic Distribuition in una lista")
print(temp[:5])

Converto la colonna Topic Distribuition in una lista
['[(0, 0.020833334), (1, 0.5208332), (2, 0.020833334), (3, 0.020833334), (4, 0.020833334), (5, 0.020833334), (6, 0.020833334), (7, 0.35416678)]', '[(0, 0.0625), (1, 0.0625), (2, 0.0625), (3, 0.0625), (4, 0.0625), (5, 0.5625), (6, 0.0625), (7, 0.0625)]', '[(0, 0.03125), (1, 0.53114474), (2, 0.03125), (3, 0.031355355), (4, 0.2812499), (5, 0.03125), (6, 0.03125), (7, 0.03125)]', '[(0, 0.008928572), (1, 0.232126), (2, 0.36607176), (3, 0.28570276), (4, 0.08035694), (5, 0.008928572), (6, 0.0089470735), (7, 0.008938288)]', '[(0, 0.3750001), (1, 0.041666668), (2, 0.041666668), (3, 0.3749999), (4, 0.041666668), (5, 0.041666668), (6, 0.041666668), (7, 0.041666668)]']


In order to convert each string into a proper list of floats I am using the 'strToList' function

In [5]:
x = [strToList(x) for x in temp ]
print("Conversione di temp in una lista di distribuzioni di probabilità")
print(x[:10])

#ho aggiunto questa conversione perchè sospetto che il fatto di averlo avuto sottoforma di lista abbia inficiato l'accuratezza
x=np.array([np.array(xi) for xi in x])
print(type(x))

Conversione di temp in una lista di distribuzioni di probabilità
[[0.020833334, 0.5208332, 0.020833334, 0.020833334, 0.020833334, 0.020833334, 0.020833334], [0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.5625, 0.0625], [0.03125, 0.53114474, 0.03125, 0.031355355, 0.2812499, 0.03125, 0.03125], [0.008928572, 0.232126, 0.36607176, 0.28570276, 0.08035694, 0.008928572, 0.0089470735], [0.3750001, 0.041666668, 0.041666668, 0.3749999, 0.041666668, 0.041666668, 0.041666668], [0.34375042, 0.17708302, 0.09375009, 0.010416666, 0.09374999, 0.010416666, 0.093749575], [0.5114528, 0.03126889, 0.031254027, 0.3010243, 0.03125, 0.03125, 0.03125], [0.28124994, 0.03125, 0.031261522, 0.03125, 0.53123856, 0.03125, 0.03125], [0.015625, 0.14062455, 0.26562533, 0.26562467, 0.015625, 0.015625, 0.14062524], [0.51040554, 0.01042255, 0.010420013, 0.010422617, 0.17707926, 0.09375002, 0.17708336]]
<class 'numpy.ndarray'>


I need to obtain the labels too in a separate vector. I do so by saving them into the vector called 'y'

In [6]:
y = data.category.tolist()
print("print della lista y contenente le categorie")
y=np.array([np.array(xi) for xi in y])
print(type(y))
print(y)

print della lista y contenente le categorie
<class 'numpy.ndarray'>
['Internet' 'Internet' 'Internet' ... 'Reading' 'Reading' 'Reading']


Here I wanted to make sure that I had the same number of labels as the number of the probability distribuitions

In [7]:
print("numero di distribuzioni di probabilità: "+str(len(x)))
print("numero di label: "+str(len(y)))

numero di distribuzioni di probabilità: 253071
numero di label: 253071


That's a first attempt in creating a new cleaned version of the original Dataset where TopicDistribuitions column had not strings but actual vectors of floats

In [13]:
#questo codice funziona quando x e y non sono di tipo ndarray, ma adesso che li ho convertiti in questo tipo
#la colonna TopicDistribuitions non me la da più correttamente, ma questo poco importa perchè non uso newDataFrame
#questo codice faceva soltanto parte di una mia congettura per capire come aumentare l'accuracy.
my_column_names=['Reviews','TopicDistribuitions']
newDataFrame=pd.DataFrame(columns=my_column_names)
newDataFrame['Reviews']=y
newDataFrame['TopicDistribuitions']=x
newDataFrame.head()

,Reviews,TopicDistribuitions
0,Internet,0.020833
1,Internet,0.062500
2,Internet,0.031250
3,Internet,0.008929
4,Internet,0.375000


I suspected that in order to train the model properly I had to explicitate each Topic as a feature in a separate column. That's why I created another DataSet from the previously DataSet

In [86]:
my_column_names=['Topic0','Topic1','Topic2','Topic3','Topic4','Topic5','Topic6']
new_DataFrame=pd.DataFrame(x,columns=my_column_names)
new_DataFrame['Category']=y

new_DataFrame=pd.get_dummies(new_DataFrame)
new_DataFrame.head()
#new_DataFrame.corr() #per generare la matrice di correlazione

features=new_DataFrame[["Topic0", "Topic1", "Topic2", "Topic3", "Topic4", "Topic5", "Topic6"]].to_numpy()
lables=new_DataFrame[["Category_Connectivity", "Category_Games", "Category_Internet", "Category_Multimedia", "Category_Navigation", "Category_Reading", "Category_Security","Category_System","Category_Theming"]].to_numpy()


[[0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


Actual splitting into a test set, training set (and validation set?)

In [75]:
X_train, X_test, y_train, y_test = train_test_split(features, lables, test_size=0.2, random_state=1) # 70% training set and 30% test set

Z_train, Z_test, W_train, W_test = train_test_split(X_train, y_train, test_size=0.4, random_state=1) # 70% training set subset and 30% validation set

Creating a Decision tree classifier object

In [76]:
clf = DecisionTreeClassifier(criterion="gini",max_depth=10)

Now I train the decision tree classifier

In [77]:
clf = clf.fit(Z_train,W_train)

Now I test the model on the test set held out in the beginning

In [78]:
y_pred = clf.predict(Z_test)

Now I measure model accuracy which means how often the classifier is correct

In [83]:
print("Accuracy:",metrics.accuracy_score(W_test, y_pred)*100,str("%"))
report = classification_report(W_test, y_pred)
print(report)

Accuracy: 31.70171517478977 %
              precision    recall  f1-score   support

           0       0.33      0.00      0.00       888
           1       0.62      0.03      0.06      7960
           2       0.55      0.02      0.03     15272
           3       0.22      0.00      0.01      6071
           4       0.59      0.72      0.65     34890
           5       0.00      0.00      0.00      2664
           6       0.00      0.00      0.00      2244
           7       0.36      0.01      0.02      9327
           8       0.20      0.00      0.00      1667

   micro avg       0.58      0.32      0.41     80983
   macro avg       0.32      0.09      0.08     80983
weighted avg       0.48      0.32      0.29     80983
 samples avg       0.32      0.32      0.32     80983



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [59]:
#con kfold e usando la metrica di accuracy
kfold = model_selection.KFold(n_splits=10, random_state=10, shuffle=True)
scoring_0 = 'accuracy' #accuracy
results_0 = model_selection.cross_val_score(LogisticRegression(solver='liblinear'), x, y, cv=kfold, scoring=scoring_0)
print("accuracy: %.3f (%.3f)" % (results_0.mean(), results_0.std()))


accuracy: 0.438 (0.003)
